- iterate over the cells
- save hydrology and sediments
- make sure to save the ls, sc

1. Experiment 1: NO GLACIERS
2. Experiment 2: transport limited case (!!! HERE: calculate what is the mean annual sedimant yeild) 
3. Experiment 3: linear increase in sediment storage: (HOW BIG LANDSLIDE SHOULD BE?)

-- to make changes in the model go to run.py, SedCas_glacier_sed.py (model) or modules.py (modules)

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt


# modelling part
from run import run


###  transport limited case (=sediment unlimited): calculate how much sediment there MAY be

In [ ]:
%%time
# climatepath = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangTL/default_land_cover'
climatepath = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/2bagrotTL/default_land_cover'
# climatepath = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/3mustangTL/default_land_cover'


# get all folders togetehr 
# ls_ = os.listdir('./langtang')
ls_ = os.listdir(climatepath)
ls = [d for d in ls_ if not d.startswith('.')]
dir0 = os.getcwd()
# print(dir0)
# run the model for each
for d in ls:
    # path = os.path.join(dir0, '1langtangTL/default_land_cover', d)
    path = os.path.join(dir0, '2bagrotTL/default_land_cover', d)
    path = os.path.join(dir0, '3mustangTL/default_land_cover', d)
    if path.endswith('.csv'):
        continue
    # print(path)
    os.chdir(path)
    run()
    os.chdir(dir0)


# how much sediment is exported per year in the transport limited case (= unlimited amount of sedoments) ?

In [ ]:
# for each output sediment folder calculate the total annual 
# go over each folder 
# take Sediment out file 
# calculate annual mean
# save into the table, where the table has: cellnr, mean annual sed yeald

In [2]:

# Define your custom function
def annual_sum_mean(df):
    df['D'] = pd.to_datetime(df['D'])
    df['Year'] = df['D'].dt.year
    annual_sum = df.groupby('Year').sum()
    annual_mean_Qstl = annual_sum.Qstl.mean()
    return annual_mean_Qstl

def annual_sum_percentile(df, percentile):
    df['D'] = pd.to_datetime(df['D'])
    df['Year'] = df['D'].dt.year
    annual_sum = df.groupby('Year').sum()
    annual_percentile_Qstl = annual_sum.Qstl.quantile(percentile / 100)
    return annual_percentile_Qstl


In [5]:
%%time
# 1- langtang
# 2 - bagrot
# 3 - mustang

# Path to the directory containing your folders
# directory_path = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangTL/default_land_cover'
directory_path = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/2bagrotTL/default_land_cover'

mean_annual_sum =[]

mean_annual_10 = []
mean_annual_20 = []
mean_annual_25 = []
mean_annual_30 = []
mean_annual_40 = []
mean_annual_50 = []
mean_annual_60 = []
mean_annual_70 = []
mean_annual_75 = []
mean_annual_80 = []
mean_annual_90 = []
mean_annual_100 = []


# Iterate over each folder
for folder_name in os.listdir(directory_path):
    folder_path = os.path.join(directory_path, folder_name)
    
    # Check if it's a directory and its name starts with 'cellnr'
    if os.path.isdir(folder_path): #and folder_name.startswith('cellnr'):
        # Locate the 'sediment.out' file within the folder
        sediment_out_path = os.path.join(folder_path, 'Sediment.out')
        
        # Read the contents of the file into a pandas DataFrame
        df = pd.read_csv(sediment_out_path, delimiter=',')  # Adjust delimiter if needed
        # mean:
        mean_annual_sum_value = annual_sum_mean(df)
        # percentiles:
        mean_annual_10_value = annual_sum_percentile(df, 10)
        mean_annual_20_value = annual_sum_percentile(df, 20)
        mean_annual_25_value = annual_sum_percentile(df, 25)
        mean_annual_30_value = annual_sum_percentile(df, 30)
        mean_annual_40_value = annual_sum_percentile(df, 40)
        mean_annual_50_value = annual_sum_percentile(df, 50)
        mean_annual_60_value = annual_sum_percentile(df, 60)
        mean_annual_70_value = annual_sum_percentile(df, 70)
        mean_annual_75_value = annual_sum_percentile(df, 75)
        mean_annual_80_value = annual_sum_percentile(df, 80)
        mean_annual_90_value = annual_sum_percentile(df, 90)
        mean_annual_100_value =  annual_sum_percentile(df, 100)

       # # Save the DataFrame into the results table
        mean_annual_sum.append((folder_name.replace('cellnr', ''), mean_annual_sum_value))

        mean_annual_10.append((folder_name.replace('cellnr', ''), mean_annual_10_value ))
        mean_annual_20.append((folder_name.replace('cellnr', ''), mean_annual_20_value ))
        mean_annual_25.append((folder_name.replace('cellnr', ''), mean_annual_25_value ))
        mean_annual_30.append((folder_name.replace('cellnr', ''), mean_annual_30_value ))
        mean_annual_40.append((folder_name.replace('cellnr', ''), mean_annual_40_value ))
        mean_annual_50.append((folder_name.replace('cellnr', ''), mean_annual_50_value ))
        mean_annual_60.append((folder_name.replace('cellnr', ''), mean_annual_60_value ))
        mean_annual_70.append((folder_name.replace('cellnr', ''), mean_annual_70_value ))
        mean_annual_75.append((folder_name.replace('cellnr', ''), mean_annual_75_value ))
        mean_annual_80.append((folder_name.replace('cellnr', ''), mean_annual_80_value ))
        mean_annual_90.append((folder_name.replace('cellnr', ''), mean_annual_90_value ))
        mean_annual_100.append((folder_name.replace('cellnr', ''),mean_annual_100_value))



# make table 
mean_annual_sum_df = pd.DataFrame(mean_annual_sum, columns=['cellnr', 'annual_mean_Qstl_mm']).set_index('cellnr')
mean_annual_10_df  = pd.DataFrame(mean_annual_10, columns=['cellnr', 'annual_10percent_Qstl_mm']).set_index('cellnr')
mean_annual_20_df  = pd.DataFrame(mean_annual_20, columns=['cellnr', 'annual_20percent_Qstl_mm']).set_index('cellnr')
mean_annual_25_df  = pd.DataFrame(mean_annual_25, columns=['cellnr', 'annual_25percent_Qstl_mm']).set_index('cellnr')
mean_annual_30_df  = pd.DataFrame(mean_annual_30, columns=['cellnr', 'annual_30percent_Qstl_mm']).set_index('cellnr')
mean_annual_40_df  = pd.DataFrame(mean_annual_40, columns=['cellnr', 'annual_40percent_Qstl_mm']).set_index('cellnr')
mean_annual_50_df  = pd.DataFrame(mean_annual_50, columns=['cellnr', 'annual_50percent_Qstl_mm']).set_index('cellnr')
mean_annual_60_df  = pd.DataFrame(mean_annual_60, columns=['cellnr', 'annual_60percent_Qstl_mm']).set_index('cellnr')
mean_annual_70_df  = pd.DataFrame(mean_annual_70, columns=['cellnr', 'annual_70percent_Qstl_mm']).set_index('cellnr')
mean_annual_75_df  = pd.DataFrame(mean_annual_75, columns=['cellnr', 'annual_75percent_Qstl_mm']).set_index('cellnr')
mean_annual_80_df  = pd.DataFrame(mean_annual_80, columns=['cellnr', 'annual_80percent_Qstl_mm']).set_index('cellnr')
mean_annual_90_df  = pd.DataFrame(mean_annual_90, columns=['cellnr', 'annual_90percent_Qstl_mm']).set_index('cellnr') 
mean_annual_100_df = pd.DataFrame(mean_annual_100, columns=['cellnr', 'annual_100percent_Qstl_mm']).set_index('cellnr')


# merge together
merged_df = pd.concat([mean_annual_sum_df, mean_annual_10_df, mean_annual_20_df,
                       mean_annual_25_df, mean_annual_30_df, mean_annual_40_df, mean_annual_50_df,
                       mean_annual_60_df, mean_annual_70_df, mean_annual_75_df, mean_annual_80_df, 
                       mean_annual_90_df, mean_annual_100_df], axis = 1)




CPU times: user 41.6 s, sys: 695 ms, total: 42.3 s
Wall time: 42.4 s


In [6]:
# len(merged_df.columns)
merged_df.head()

,annual_mean_Qstl_mm,annual_10percent_Qstl_mm,annual_20percent_Qstl_mm,annual_25percent_Qstl_mm,annual_30percent_Qstl_mm,annual_40percent_Qstl_mm,annual_50percent_Qstl_mm,annual_60percent_Qstl_mm,annual_70percent_Qstl_mm,annual_75percent_Qstl_mm,annual_80percent_Qstl_mm,annual_90percent_Qstl_mm,annual_100percent_Qstl_mm
cellnr,,,,,,,,,,,,,
16a,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9b,0.340248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,21.287030
10c,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
10d,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
15a,0.273555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.116678


In [ ]:
# mean_annual_sum_df['annual_mean_Qstl_m'] = mean_annual_sum_df.annual_mean_Qstl_mm/1000     # [sediments mm to m] 
# mean_annual_sum_df['annual_mean_Qstl_m3'] = mean_annual_sum_df.annual_mean_Qstl_m * 4.83*(10**6)   # [sediments [m] * area [m2]
# merged_df.sort_values(merged_df.index)




In [7]:
for column in merged_df.columns:
    # for the columns with _mm in it
    if '_mm' in column:
        # Replace '_mm' with '_m3' and create a new column
        new_column_name = column.replace('_mm', '_m3')
        # convert to m3: 1) [sediments mm to m] 2) [sediments [m] * area [m2]
        merged_df[new_column_name] = (merged_df[column] / 1000) * 4.83 * (10 ** 6)



In [8]:
# merged_df

## m3/day

In [9]:
# Iterate over each column in the DataFrame
for column in merged_df.columns:
    if '_m3' in column:
        merged_df[column + '_day'] = merged_df[column] /365


In [10]:
merged_df.head()



,annual_mean_Qstl_mm,annual_10percent_Qstl_mm,annual_20percent_Qstl_mm,annual_25percent_Qstl_mm,annual_30percent_Qstl_mm,annual_40percent_Qstl_mm,annual_50percent_Qstl_mm,annual_60percent_Qstl_mm,annual_70percent_Qstl_mm,annual_75percent_Qstl_mm,...,annual_25percent_Qstl_m3_day,annual_30percent_Qstl_m3_day,annual_40percent_Qstl_m3_day,annual_50percent_Qstl_m3_day,annual_60percent_Qstl_m3_day,annual_70percent_Qstl_m3_day,annual_75percent_Qstl_m3_day,annual_80percent_Qstl_m3_day,annual_90percent_Qstl_m3_day,annual_100percent_Qstl_m3_day
cellnr,,,,,,,,,,,,,,,,,,,,,
16a,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9b,0.340248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,281.688641
10c,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
10d,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
15a,0.273555,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,173.571380


# add coordinates and save the output 

In [11]:
# csv with coordinates and geopotential (z) 

# coordinates = pd.read_csv('/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_with_labels_langtang.csv').set_index('cellnr2')
coordinates = pd.read_csv('/Users/varyabazilova/Desktop/paper2/downscaling_simple/coordinates_with_labels_bagrot.csv').set_index('cellnr2')
with_coords = pd.concat([merged_df, coordinates],  axis=1).reset_index()
# coordinates



In [12]:
# with_coords

In [13]:
#save alltogether 
# output_path = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangTL/default_land_cover/'
output_path = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/2bagrotTL/default_land_cover/'
with_coords.to_csv(output_path + 'default_land_cover_bagrot_tl_percentiles_with_coords.csv', index = False)



In [14]:
with_coords.columns

Index(['index', 'annual_mean_Qstl_mm', 'annual_10percent_Qstl_mm',
       'annual_20percent_Qstl_mm', 'annual_25percent_Qstl_mm',
       'annual_30percent_Qstl_mm', 'annual_40percent_Qstl_mm',
       'annual_50percent_Qstl_mm', 'annual_60percent_Qstl_mm',
       'annual_70percent_Qstl_mm', 'annual_75percent_Qstl_mm',
       'annual_80percent_Qstl_mm', 'annual_90percent_Qstl_mm',
       'annual_100percent_Qstl_mm', 'annual_mean_Qstl_m3',
       'annual_10percent_Qstl_m3', 'annual_20percent_Qstl_m3',
       'annual_25percent_Qstl_m3', 'annual_30percent_Qstl_m3',
       'annual_40percent_Qstl_m3', 'annual_50percent_Qstl_m3',
       'annual_60percent_Qstl_m3', 'annual_70percent_Qstl_m3',
       'annual_75percent_Qstl_m3', 'annual_80percent_Qstl_m3',
       'annual_90percent_Qstl_m3', 'annual_100percent_Qstl_m3',
       'annual_mean_Qstl_m3_day', 'annual_10percent_Qstl_m3_day',
       'annual_20percent_Qstl_m3_day', 'annual_25percent_Qstl_m3_day',
       'annual_30percent_Qstl_m3_day', 'ann

In [15]:
columns_to_copy = ['cellnr', 'annual_10percent_Qstl_m3_day',
                   'annual_20percent_Qstl_m3_day', 'annual_25percent_Qstl_m3_day',
                   'annual_30percent_Qstl_m3_day', 'annual_40percent_Qstl_m3_day',
                   'annual_50percent_Qstl_m3_day', 'annual_60percent_Qstl_m3_day',
                   'annual_70percent_Qstl_m3_day', 'annual_75percent_Qstl_m3_day',
                   'annual_80percent_Qstl_m3_day', 'annual_90percent_Qstl_m3_day',
                   'annual_100percent_Qstl_m3_day', 'Longitude', 'Latitude',
                   'Longitude_new', 'Latitude_new', 'cellnr', 'z']

selected_columns= with_coords[columns_to_copy].copy()
                              

In [16]:

selected_columns.to_csv(output_path + 'default_land_cover_bagrot_tl_percentiles_with_coords_m3_day.csv', index = False)

## supply limited case runs


In [ ]:
%%time

# default land cover 
# climatepath = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangSL/default_land_cover/1default_land_cover_mean_annual'
# climatepath = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangSL/default_land_cover/2default_land_cover_25percent'
# climatepath = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangSL/default_land_cover/3default_land_cover_50percent'
# climatepath = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangSL/default_land_cover/4default_land_cover_75percent'


# 10percent glacier cover
# climatepath = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangSL/10percent_glacier/110percent_glacier_mean_annual'
# climatepath = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangSL/10percent_glacier/210percent_glacier_25percent'
# climatepath = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangSL/10percent_glacier/310percent_glacier_50percent'
climatepath = '/Users/varyabazilova/Desktop/paper2/202405_modelruns/1langtangSL/10percent_glacier/410percent_glacier_75percent'




# get all folders togetehr 
# ls_ = os.listdir('./langtang')
ls_ = os.listdir(climatepath)
ls = [d for d in ls_ if not d.startswith('.')]
dir0 = os.getcwd()

# run the model for each
for d in ls:
    # path = os.path.join(dir0, 'TL/default_land_cover', d)
    # path = os.path.join(dir0, '1langtangSL/default_land_cover/1default_land_cover_mean_annual', d)
    # path = os.path.join(dir0, '1langtangSL/default_land_cover/2default_land_cover_25percent', d)
    # path = os.path.join(dir0, '1langtangSL/default_land_cover/3default_land_cover_50percent', d)
    # path = os.path.join(dir0, '1langtangSL/default_land_cover/4default_land_cover_75percent', d)

    # 10percent glacier cover
    # path = os.path.join(dir0, '1langtangSL/10percent_glacier/110percent_glacier_mean_annual', d)
    # path = os.path.join(dir0, '1langtangSL/10percent_glacier/210percent_glacier_25percent', d)
    # path = os.path.join(dir0, '1langtangSL/10percent_glacier/310percent_glacier_50percent', d)
    # path = os.path.join(dir0, '1langtangSL/10percent_glacier/410percent_glacier_75percent', d)
    print(path)
    os.chdir(path)
    run()
    os.chdir(dir0)


In [ ]:
os.getcwd()

## 